In [1]:
from sympy import *
import regex

pax, pay, pbx, pby, vax, vay, vbx, vby, aax, aay, abx, aby, pix, piy = symbols('p_a_x p_a_y p_b_x p_b_y v_a_x v_a_y v_b_x v_b_y a_a_x a_a_y a_b_x a_b_y p_i_x p_i_y',real=True)
F, T = symbols('F T',real=True,positive=True)
Tr, Ti = symbols('T_r T_i', real=True)


def gdcode(expr):
    expr = collect_const(expr)
    rhs = ccode(expr)
    opc = count_ops(expr)
    # turn squares into multiplication
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*2\s*\)', r'(\1*\1)', rhs)
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*-2\s*\)', r'(1.0/(\1*\1))', rhs)
    # turn cubes into multiplication
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*3\s*\)', r'(\1*\1*\1)', rhs)
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*-3\s*\)', r'(1.0/(\1*\1*\1))', rhs)
    # convert integers to floats
    rhs = regex.sub(r'(?<!\.)\b(\d+)\b(?!\.)',r'\1.0',rhs)
    # rename vector parts
    rhs = regex.sub(r'\b(\w+)_(x|y)',r'\1.\2',rhs)
    # fix ternary if
    rhs = regex.sub(r'\(\s*\(\s*(.*?)\s*\)\s*\?\s*\(\s*\n\s*(.*?)\s*\n\s*\)\s*\n\s*:\s*\(\s*\n\s*(.*?)\s*\n\s*\)\s*\)', r'\2 if \1 else \3', rhs)
    # fix fabs
    rhs = regex.sub(r'fabs',r'abs',rhs)
    return (rhs,opc)

def make_code(qs):
    cse_qs = cse(qs,symbols=numbered_symbols('_'))
    qs = cse_qs[0] + [(q.lhs,q.rhs) for q in cse_qs[1]]
    code=''
    opcount=0
    for q in qs:
        rhs, opc = gdcode(q[1])
        opcount+=opc
        code+='var '+str(q[0])+':float = '+rhs
        code+='\n'
    
    code = '# opcount: '+str(opcount)+'\n'+code
    code = '# generated by ballistics_gen.ipynb\n' + code

    return code


def solve_quart(a,b,c,d,e):
    p1 = 2*c**3 - 9*b*c*d + 27*a*d**2 + 27*e*b**2 - 72*a*c*e
    p2 = p1 + sqrt(-4*(c**2 - 3*b*d + 12*a*e)**3 + p1**2)
    p3 = (c**2 - 3*b*d + 12*a*e)/(3*a*cbrt(p2/2)) + cbrt(p2/2)/(3*a)
    p4 = sqrt(b**2/(4*a**2) - 2*c/(3*a) + p3)
    p5 = b**2/(2*a**2) - 4*c/(3*a) - p3
    p6 = (-b**3/(a*3) + 4*b*c/(a**2) - 8*d/a)/(4*p4)
    return [
        -b/(4*a) - p4/2 - sqrt(p5-p6)/2,
        -b/(4*a) - p4/2 + sqrt(p5-p6)/2,
        -b/(4*a) + p4/2 - sqrt(p5+p6)/2,
        -b/(4*a) + p4/2 + sqrt(p5+p6)/2
    ]

In [2]:
# quartic solver
a, b, c, d, e = symbols('a b c d e',real=True)

qs = solve(a*T**4 + b*T**3 + c*T**2 + d*T + e, T)
sols = []
for q in qs:
    ri = [piecewise_fold(x) for x in q.as_real_imag()]
    sols += [Eq(Tr,ri[0]),Eq(Ti,ri[1])]
print(make_code(sols))
display(sols)

# generated by ballistics_gen.ipynb
# opcount: 268
var _0:float = 1.0/a
var _1:float = _0*c
var _2:float = (1.0/(a*a))
var _3:float = (b*b)
var _4:float = _2*_3
var _5:float = _1 - 3.0/8.0*_4
var _6:float = (_5*_5*_5)
var _7:float = _0*d
var _8:float = _2*b
var _9:float = _8*c
var _10:float = (1.0/(a*a*a))
var _11:float = _10*(b*b*b)
var _12:float = pow((1.0/8.0)*_11 + _7 - 1.0/2.0*_9, 2.0)
var _13:float = _0*e
var _14:float = _8*d
var _15:float = _13 - 1.0/4.0*_14
var _16:float = (1.0/16.0)*_10*_3*c + _15 - 3.0/256.0*pow(b, 4.0)/pow(a, 4.0)
var _17:float = -1.0/8.0*_12 + (1.0/3.0)*_16*_5 - 1.0/108.0*_6
var _18:float = (1.0/3.0)*atan2(0.0, _17)
var _19:float = sin(_18)
var _20:float = (_17*_17)
var _21:float = 2.0*pow(_20, 1.0/6.0)
var _22:float = _19*_21
var _23:float = (2.0/3.0)*_1
var _24:float = _21*cos(_18)
var _25:float = (1.0/4.0)*_2*_3 - (_23 + _24)
var _26:float = (1.0/2.0)*atan2(-_22, _25)
var _27:float = sin(_26)
var _28:float = pow(4.0*(_19*_19)*cbrt(_20) + (_25*_25), 1.0/4

[Eq(T_r, Piecewise((-((2*((-(c/a - 3*b**2/(8*a**2))**3/108 + (c/a - 3*b**2/(8*a**2))*(e/a - b*d/(4*a**2) + b**2*c/(16*a**3) - 3*b**4/(256*a**4))/3 - (d/a - b*c/(2*a**2) + b**3/(8*a**3))**2/8)**2)**(1/6)*sin(atan2(0, -(c/a - 3*b**2/(8*a**2))**3/108 + (c/a - 3*b**2/(8*a**2))*(e/a - b*d/(4*a**2) + b**2*c/(16*a**3) - 3*b**4/(256*a**4))/3 - (d/a - b*c/(2*a**2) + b**3/(8*a**3))**2/8)/3) - (2*d/a - b*c/a**2 + b**3/(4*a**3))*sin(atan2(-2*((-(c/a - 3*b**2/(8*a**2))**3/108 + (c/a - 3*b**2/(8*a**2))*(e/a - b*d/(4*a**2) + b**2*c/(16*a**3) - 3*b**4/(256*a**4))/3 - (d/a - b*c/(2*a**2) + b**3/(8*a**3))**2/8)**2)**(1/6)*sin(atan2(0, -(c/a - 3*b**2/(8*a**2))**3/108 + (c/a - 3*b**2/(8*a**2))*(e/a - b*d/(4*a**2) + b**2*c/(16*a**3) - 3*b**4/(256*a**4))/3 - (d/a - b*c/(2*a**2) + b**3/(8*a**3))**2/8)/3), -2*((-(c/a - 3*b**2/(8*a**2))**3/108 + (c/a - 3*b**2/(8*a**2))*(e/a - b*d/(4*a**2) + b**2*c/(16*a**3) - 3*b**4/(256*a**4))/3 - (d/a - b*c/(2*a**2) + b**3/(8*a**3))**2/8)**2)**(1/6)*cos(atan2(0, -(c/a - 3*b*

In [3]:
# linear-linear intercept
q_pix = Eq(vax*T, vbx*T + pbx)
q_piy = Eq(vay*T, vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, vax**2 + vay**2)
display(q_f)

q_vax = Eq(vax,solve(q_pix, vax)[0])
q_vay = Eq(vay,solve(q_piy, vay)[0])
display(q_vax, q_vay)

q_f2 = q_f.subs({vax:q_vax.rhs, vay:q_vay.rhs})
display(q_f2)
q_f2 = Eq(q_f2.lhs * 4*T**2, q_f2.rhs * 4*T**2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
#q_ca = Eq(ca, q_f2_terms[T**4])
#q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_cc, q_cd, q_ce)
#q_f2 = q_f2.subs({q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [Eq(T,q.subs({q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs})) for q in q_t]
print(make_code(sols))
print(make_code([q_vax,q_vay]))

Eq(T*v_a_x, T*v_b_x + p_b_x)

Eq(T*v_a_y, T*v_b_y + p_b_y)

Eq(F**2, v_a_x**2 + v_a_y**2)

Eq(v_a_x, v_b_x + p_b_x/T)

Eq(v_a_y, v_b_y + p_b_y/T)

...

Eq(c_d, -8*p_b_x*v_b_x - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_b_x**2 - 4*p_b_y**2)

T**2*(4*F**2 - 4*v_b_x**2 - 4*v_b_y**2) + T*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y) - 4*p_b_x**2 - 4*p_b_y**2

[(-p_b_x*v_b_x - p_b_y*v_b_y + sqrt(F**2*p_b_x**2 + F**2*p_b_y**2 - p_b_x**2*v_b_y**2 + 2*p_b_x*p_b_y*v_b_x*v_b_y - p_b_y**2*v_b_x**2))/(-F**2 + v_b_x**2 + v_b_y**2),
 (p_b_x*v_b_x + p_b_y*v_b_y + sqrt(F**2*p_b_x**2 + F**2*p_b_y**2 - p_b_x**2*v_b_y**2 + 2*p_b_x*p_b_y*v_b_x*v_b_y - p_b_y**2*v_b_x**2))/(F**2 - v_b_x**2 - v_b_y**2)]

# generated by ballistics_gen.ipynb
# opcount: 27
var _0:float = (v_b.x*v_b.x)
var _1:float = (v_b.y*v_b.y)
var _2:float = (F*F)
var _3:float = _0 + _1 - _2
var _4:float = p_b.x*v_b.x
var _5:float = p_b.y*v_b.y
var _6:float = (p_b.x*p_b.x)
var _7:float = (p_b.y*p_b.y)
var _8:float = sqrt(-_0*_7 - _1*_6 + _2*_6 + _2*_7 + 2.0*_4*_5)
var _9:float = _4 + _5
var T:float = (_8 - _9)/_3
var T:float = -(_8 + _9)/_3

# generated by ballistics_gen.ipynb
# opcount: 5
var _0:float = 1.0/T
var v_a_x:float = _0*p_b.x + v_b.x
var v_a_y:float = _0*p_b.y + v_b.y



In [4]:
# linear-quadratic intercept
q_pix = Eq(vax*T, abx*(T**2)/2 + vbx*T + pbx)
q_piy = Eq(vay*T, aby*(T**2)/2 + vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, vax**2 + vay**2)
display(q_f)

q_vax = Eq(vax,solve(q_pix, vax)[0])
q_vay = Eq(vay,solve(q_piy, vay)[0])
display(q_vax, q_vay)

q_f2 = q_f.subs({vax:q_vax.rhs, vay:q_vay.rhs})
display(q_f2)
display(expand(q_f2))
q_f2 = Eq(q_f2.lhs * 4*T**2, q_f2.rhs * 4*T**2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)

print(make_code([Eq(a,q_f2_terms[T**4]), Eq(b,q_f2_terms[T**3]), Eq(c,q_f2_terms[T**2]), Eq(d,q_f2_terms[T**1]), Eq(e,q_f2_terms[1])]))
print('var T:float = solve_quart(a,b,c,d,e)\n')
print(make_code([q_vax,q_vay,Eq(pix,q_pix.rhs),Eq(piy,q_piy.rhs)]))

Eq(T*v_a_x, T**2*a_b_x/2 + T*v_b_x + p_b_x)

Eq(T*v_a_y, T**2*a_b_y/2 + T*v_b_y + p_b_y)

Eq(F**2, v_a_x**2 + v_a_y**2)

Eq(v_a_x, T*a_b_x/2 + v_b_x + p_b_x/T)

Eq(v_a_y, T*a_b_y/2 + v_b_y + p_b_y/T)

Eq(F**2, (T*a_b_x/2 + v_b_x + p_b_x/T)**2 + (T*a_b_y/2 + v_b_y + p_b_y/T)**2)

Eq(F**2, T**2*a_b_x**2/4 + T**2*a_b_y**2/4 + T*a_b_x*v_b_x + T*a_b_y*v_b_y + a_b_x*p_b_x + a_b_y*p_b_y + v_b_x**2 + v_b_y**2 + 2*p_b_x*v_b_x/T + 2*p_b_y*v_b_y/T + p_b_x**2/T**2 + p_b_y**2/T**2)

4*F**2*T**2 - T**2*(4*(T*a_b_x/2 + v_b_x + p_b_x/T)**2 + 4*(T*a_b_y/2 + v_b_y + p_b_y/T)**2)

T**4*(-a_b_x**2 - a_b_y**2) + T**3*(-4*a_b_x*v_b_x - 4*a_b_y*v_b_y) + T**2*(4*F**2 - 4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x**2 - 4*v_b_y**2) + T*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y) - 4*p_b_x**2 - 4*p_b_y**2

# generated by ballistics_gen.ipynb
# opcount: 30
var _0:float = 4.0*a_b.x
var _1:float = 4.0*a_b.y
var a:float = -((a_b.x*a_b.x) + (a_b.y*a_b.y))
var b:float = -(_0*v_b.x + _1*v_b.y)
var c:float = 4.0*(F*F) - (_0*p_b.x + _1*p_b.y + 4.0*(v_b.x*v_b.x) + 4.0*(v_b.y*v_b.y))
var d:float = -8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var e:float = -4.0*((p_b.x*p_b.x) + (p_b.y*p_b.y))

var T:float = solve_quart(a,b,c,d,e)

# generated by ballistics_gen.ipynb
# opcount: 20
var _0:float = (1.0/2.0)*T
var _1:float = 1.0/T
var _2:float = (1.0/2.0)*(T*T)
var v_a_x:float = _0*a_b.x + _1*p_b.x + v_b.x
var v_a_y:float = _0*a_b.y + _1*p_b.y + v_b.y
var p_i_x:float = T*v_b.x + _2*a_b.x + p_b.x
var p_i_y:float = T*v_b.y + _2*a_b.y + p_b.y



In [5]:
# quadratic-linear intercept

q_pix = Eq(aax*(T**2)/2 , vbx*T + pbx)
q_piy = Eq(aay*(T**2)/2 , vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, aax**2 + aay**2)
display(q_f)

q_aax = Eq(aax,solve(q_pix, aax)[0])
q_aay = Eq(aay,solve(q_piy, aay)[0])
display(q_aax, q_aay)

q_f2 = q_f.subs({aax:q_aax.rhs, aay:q_aay.rhs})
q_f2 = Eq(q_f2.lhs * T**4, q_f2.rhs * T**4)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)

print(make_code([Eq(a,q_f2_terms[T**4]), Eq(b,0), Eq(c,q_f2_terms[T**2]), Eq(d,q_f2_terms[T**1]), Eq(e,q_f2_terms[1])]))
print('var T:float = solve_quart(a,b,c,d,e)\n')
print(make_code([q_aax,q_aay,Eq(pix,q_pix.rhs),Eq(piy,q_piy.rhs)]))

Eq(T**2*a_a_x/2, T*v_b_x + p_b_x)

Eq(T**2*a_a_y/2, T*v_b_y + p_b_y)

Eq(F**2, a_a_x**2 + a_a_y**2)

Eq(a_a_x, 2*(T*v_b_x + p_b_x)/T**2)

Eq(a_a_y, 2*(T*v_b_y + p_b_y)/T**2)

F**2*T**4 - T**4*(4*(T*v_b_x + p_b_x)**2/T**4 + 4*(T*v_b_y + p_b_y)**2/T**4)

F**2*T**4 + T**2*(-4*v_b_x**2 - 4*v_b_y**2) + T*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y) - 4*p_b_x**2 - 4*p_b_y**2

# generated by ballistics_gen.ipynb
# opcount: 16
var a:float = (F*F)
var b:float = 0.0
var c:float = -4.0*((v_b.x*v_b.x) + (v_b.y*v_b.y))
var d:float = -8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var e:float = -4.0*((p_b.x*p_b.x) + (p_b.y*p_b.y))

var T:float = solve_quart(a,b,c,d,e)

# generated by ballistics_gen.ipynb
# opcount: 8
var _0:float = T*v_b.x + p_b.x
var _1:float = 2.0/(T*T)
var _2:float = T*v_b.y + p_b.y
var a_a_x:float = _0*_1
var a_a_y:float = _1*_2
var p_i_x:float = _0
var p_i_y:float = _2



In [6]:
# quadratic-quadratic intercept

q_pix = Eq(aax*(T**2)/2, abx*(T**2)/2 + vbx*T + pbx)
q_piy = Eq(aay*(T**2)/2, aby*(T**2)/2 + vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, aax**2 + aay**2)
display(q_f)

q_aax = Eq(aax,solve(q_pix, aax)[0])
q_aay = Eq(aay,solve(q_piy, aay)[0])
display(q_aax, q_aay)

q_f2 = q_f.subs({aax:q_aax.rhs, aay:q_aay.rhs})
q_f2 = Eq(q_f2.lhs * T**4, q_f2.rhs * T**4)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)

print(make_code([Eq(a,q_f2_terms[T**4]), Eq(b,q_f2_terms[T**3]), Eq(c,q_f2_terms[T**2]), Eq(d,q_f2_terms[T**1]), Eq(e,q_f2_terms[1])]))
print('var T:float = solve_quart(a,b,c,d,e)\n')
print(make_code([q_aax,q_aay,Eq(pix,q_pix.rhs),Eq(piy,q_piy.rhs)]))

Eq(T**2*a_a_x/2, T**2*a_b_x/2 + T*v_b_x + p_b_x)

Eq(T**2*a_a_y/2, T**2*a_b_y/2 + T*v_b_y + p_b_y)

Eq(F**2, a_a_x**2 + a_a_y**2)

Eq(a_a_x, a_b_x + 2*v_b_x/T + 2*p_b_x/T**2)

Eq(a_a_y, a_b_y + 2*v_b_y/T + 2*p_b_y/T**2)

F**2*T**4 - T**4*((a_b_x + 2*v_b_x/T + 2*p_b_x/T**2)**2 + (a_b_y + 2*v_b_y/T + 2*p_b_y/T**2)**2)

T**4*(F**2 - a_b_x**2 - a_b_y**2) + T**3*(-4*a_b_x*v_b_x - 4*a_b_y*v_b_y) + T**2*(-4*a_b_x*p_b_x - 4*a_b_y*p_b_y - 4*v_b_x**2 - 4*v_b_y**2) + T*(-8*p_b_x*v_b_x - 8*p_b_y*v_b_y) - 4*p_b_x**2 - 4*p_b_y**2

# generated by ballistics_gen.ipynb
# opcount: 27
var _0:float = 4.0*a_b.x
var _1:float = 4.0*a_b.y
var a:float = (F*F) - ((a_b.x*a_b.x) + (a_b.y*a_b.y))
var b:float = -(_0*v_b.x + _1*v_b.y)
var c:float = -(_0*p_b.x + _1*p_b.y + 4.0*(v_b.x*v_b.x) + 4.0*(v_b.y*v_b.y))
var d:float = -8.0*(p_b.x*v_b.x + p_b.y*v_b.y)
var e:float = -4.0*((p_b.x*p_b.x) + (p_b.y*p_b.y))

var T:float = solve_quart(a,b,c,d,e)

# generated by ballistics_gen.ipynb
# opcount: 20
var _0:float = (T*T)
var _1:float = 2.0/_0
var _2:float = 2.0/T
var _3:float = (1.0/2.0)*_0
var a_a_x:float = _1*p_b.x + _2*v_b.x + a_b.x
var a_a_y:float = _1*p_b.y + _2*v_b.y + a_b.y
var p_i_x:float = T*v_b.x + _3*a_b.x + p_b.x
var p_i_y:float = T*v_b.y + _3*a_b.y + p_b.y

